In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np
from PIL import Image, ExifTags
import torch

In [2]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
	"path" : str(DATASETS_DIR.absolute()),
	'train': str(TRAIN_IMAGES_DIR.absolute()),
	'val': str(VAL_IMAGES_DIR.absolute()),
	'test': str(TEST_IMAGES_DIR.absolute()),
	'nc': num_classes,
	'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [3]:
val_df['Image_ID'].nunique()

56

In [4]:
from glob import glob

PATHS = [
    "zindi_challenge_cacao/train5/weights/best.pt",
	"zindi_challenge_cacao/train6/weights/best.pt",
	"zindi_challenge_cacao/train7/weights/best.pt",
]

In [5]:
# Validate the model on the validation set
CFG_PATHS = [
    "zindi_challenge_cacao/train5/args.yaml",
	"zindi_challenge_cacao/train6/args.yaml",
	"zindi_challenge_cacao/train7/args.yaml",
]

In [6]:
from MultiPredictions import MergedYOLOPredictor

# Load the trained YOLO model
model = MergedYOLOPredictor(PATHS)

Loading model: zindi_challenge_cacao/train5/weights/best.pt
Loading model: zindi_challenge_cacao/train6/weights/best.pt
Loading model: zindi_challenge_cacao/train7/weights/best.pt
Loaded 3 models.
Class mapping: {0: 'anthracnose', 1: 'cssvd', 2: 'healthy'}


In [7]:
for flag, v in ExifTags.TAGS.items():
	if v == "Orientation":
		break


def load_image_(filepath):
	image = Image.open(filepath)
	# return image

	exif = image._getexif()
	if exif is None:
		return image

	orientation_value = exif.get(flag, None)

	if orientation_value == 3:
		image = image.rotate(180, expand=True)
	elif orientation_value == 6:
		image = image.rotate(270, expand=True)
	elif orientation_value == 8:
		image = image.rotate(90, expand=True)
	return image

from ultralytics.utils.patches import imread
import cv2

def load_image(filepath):
	# return load_image_(filepath)
	return imread(filepath, cv2.IMREAD_COLOR)


flag

274

In [8]:
import yaml

cfgs: list[dict] = []
for path in CFG_PATHS:
	# Load the YAML file
	with open(path, 'r') as f:
		cfg: dict = yaml.safe_load(f)
	cfgs.append(cfg)

In [9]:
# Batch size for predictions
batch_size = 16
for cfg in cfgs:
	cfg["device"] = "cuda:1"
	cfg["batch"] = batch_size
	cfg["conf"] = 0.
	cfg["verbose"] = False

	cfg.pop("source")
	# cfg.pop("batch_size")
	cfg.pop("visualize")

	keys = list(cfg.keys())
	for col in keys:
		if "show" in col or "save" in col:
			cfg.pop(col)

In [10]:
# Path to the test images directory
test_dir_path = VAL_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = [i for i in os.listdir(test_dir_path) if not i.endswith(".npy")]

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []


with torch.no_grad():
	# Process images in batches
	for i in tqdm(range(0, len(image_files), batch_size)):
		batch_files = image_files[i:i + batch_size]
		batch_images = [load_image(os.path.join(test_dir_path, img_file)) for img_file in batch_files]

		# Make predictions on the batch of images
		results = model.predict(
			batch_images,
			cfgs,
		)

		# Iterate through each result in the batch
		for img_file, result in zip(batch_files, results):
			if result["detections"]:  # If detections are found
				for raw in result["detections"]:
					x1, y1, x2, y2 = raw["bbox"]  # Bounding boxes in xyxy format
					cls = raw["class"]  # Class indices
					conf = raw["confidence"]  # Confidence scores
					# Add the result to the all_data list
					all_data.append(
						{
							"Image_ID": str(img_file),
							"class": cls,
							"confidence": conf,
							"ymin": y1,
							"xmin": x1,
							"ymax": y2,
							"xmax": x2,
						}
					)
			else:  # If no objects are detected
				all_data.append(
					{
						"Image_ID": str(img_file),
						"class": "None",
						"confidence": None,
						"ymin": None,
						"xmin": None,
						"ymax": None,
						"xmax": None,
					}
				)


  0%|          | 0/4 [00:00<?, ?it/s]

Predicting with model for class index 0 (anthracnose)
Predicting with model for class index 1 (cssvd)
Predicting with model for class index 2 (healthy)


 25%|██▌       | 1/4 [00:05<00:17,  5.87s/it]

Predicting with model for class index 0 (anthracnose)
Predicting with model for class index 1 (cssvd)
Predicting with model for class index 2 (healthy)


 50%|█████     | 2/4 [00:08<00:07,  3.97s/it]

Predicting with model for class index 0 (anthracnose)
Predicting with model for class index 1 (cssvd)
Predicting with model for class index 2 (healthy)


 75%|███████▌  | 3/4 [00:10<00:03,  3.20s/it]

Predicting with model for class index 0 (anthracnose)
Predicting with model for class index 1 (cssvd)
Predicting with model for class index 2 (healthy)


100%|██████████| 4/4 [00:12<00:00,  3.06s/it]


In [11]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [12]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_MYSxE2.jpg,anthracnose,0.004045,0.000000,1250.661133,539.868347,1785.322998
1,ID_MYSxE2.jpg,anthracnose,0.001697,23.263847,393.226776,3264.000000,1880.456177
2,ID_MYSxE2.jpg,anthracnose,0.000538,0.000000,0.000000,214.322647,489.249329
3,ID_MYSxE2.jpg,anthracnose,0.000170,2217.458984,388.098358,3264.000000,1712.531006
4,ID_MYSxE2.jpg,anthracnose,0.000115,1.097527,741.019226,622.677979,1945.967773


In [13]:
sub.describe()

,confidence,ymin,xmin,ymax,xmax
count,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000
mean,0.014273,685.160239,609.811969,1236.185757,1099.749532
std,0.089103,936.233999,734.402532,1150.944272,896.683264
min,0.000003,0.000000,0.000000,0.000000,0.000000
25%,0.000017,0.773249,13.798838,314.744759,414.911194
50%,0.000076,274.186325,374.838791,933.242523,863.205078
75%,0.000524,1030.702362,876.130157,1797.991150,1536.000000
max,0.886526,3961.711670,3958.430664,4128.000000,4032.000000


In [14]:
sub['class'].value_counts()

class
anthracnose    1680
cssvd          1680
healthy        1680
Name: count, dtype: int64

In [15]:
def load_yolo_labels(label_folder):
	label_data = {}
	label_folder = Path(label_folder)
	paths = [i for i in label_folder.glob("*") if i.suffix != ".npy"]

	for label_file in paths:
		with open(label_file, "r") as file:
			annotations = []
			for line in file:
				parts = line.strip().split()
				if len(parts) == 5:
					class_id, x_center, y_center, width, height = map(float, parts)
					annotations.append({
						"class_id": int(class_id),
						"x_center": x_center,
						"y_center": y_center,
						"width": width,
						"height": height
					})
				else:
					print(f"Skipping line in {label_file}: {line.strip()}")
			label_data[label_file.stem] = annotations
	# Convert the label data to a pandas DataFrame
	label_df = []
	for image_id, annotations in label_data.items():
		for annotation in annotations:
			label_df.append({
				"Image_ID": image_id,
				"class_id": annotation["class_id"],
				"x_center": annotation["x_center"],
				"y_center": annotation["y_center"],
				"width": annotation["width"],
				"height": annotation["height"]
			})

	label_df = pd.DataFrame(label_df)
	return label_df

# Example usage
label_folder = VAL_LABELS_DIR
labels = load_yolo_labels(label_folder)
labels.sample(5)

def yolo_to_bbox(image_folder, labels_df: pd.DataFrame):
	image_folder = Path(image_folder)
	converted_bboxes = []

	paths = [i for i in image_folder.glob("*") if i.suffix != ".npy"]
	for image_file in paths:
		image_id = image_file.stem
		if image_id not in labels_df['Image_ID'].values:
			converted_bboxes.append({
				"Image_ID": image_id,
				"class_id": -1,  # Indicating no label
				"xmin": None,
				"ymin": None,
				"xmax": None,
				"ymax": None
			})

	for _, row in labels_df.iterrows():
		all_ids = [i for i in image_folder.glob(f"{row['Image_ID']}*") if i.suffix != ".npy"]
		image_path = image_folder / f"{row['Image_ID']}"
		if all_ids:
			image_path = all_ids[0]

		if image_path.exists():
			img = load_image_(image_path)
			img_width, img_height = img.size

			x_center = row['x_center'] * img_width
			y_center = row['y_center'] * img_height
			width = row['width'] * img_width
			height = row['height'] * img_height

			x_min = x_center - (width / 2)
			y_min = y_center - (height / 2)
			x_max = x_center + (width / 2)
			y_max = y_center + (height / 2)

			converted_bboxes.append({
				"Image_ID": row['Image_ID'],
				"class_id": row['class_id'],
				"xmin": x_min,
				"ymin": y_min,
				"xmax": x_max,
				"ymax": y_max
			})
		else:
			print(f"Image {image_path} not found.")

	return pd.DataFrame(converted_bboxes)

# Example usage
converted_labels = yolo_to_bbox(VAL_IMAGES_DIR, labels)
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax
21,ID_YQlY8q,1,0.999168,300.000256,590.999808,1079.000064
6,ID_vdbf6i,1,41.999520,33.000320,777.999840,1280.000640
79,ID_UIprDv,1,380.000256,389.999616,913.999872,1154.999296
23,ID_Xaw09O,2,0.000000,114.000000,438.000000,1279.999360
65,ID_kOXMTE,2,2415.000960,113.998464,3225.001728,547.999488


In [16]:
converted_labels.describe()

,class_id,xmin,ymin,xmax,ymax
count,95.000000,95.000000,95.000000,95.000000,95.000000
mean,1.126316,358.452658,331.631492,1269.147347,1588.589391
std,0.775175,538.727657,481.446497,950.695543,1184.230438
min,0.000000,-0.002016,-0.001632,58.999824,53.000064
25%,1.000000,1.000392,20.500032,498.499968,704.499664
50%,1.000000,114.999840,114.000000,960.000000,1195.000320
75%,2.000000,477.500328,513.499776,1862.000208,2442.499248
max,2.000000,2514.001344,2346.998784,3897.998496,4128.002064


In [17]:
converted_labels["Image_ID"].nunique()

56

In [18]:
converted_labels['class_id'].value_counts()

class_id
1    37
2    35
0    23
Name: count, dtype: int64

In [19]:
class_map

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [20]:
id_class_map = {v: k for k, v in class_map.items()}
converted_labels['class'] = converted_labels['class_id'].map(id_class_map)
converted_labels['class'].value_counts()

class
cssvd          37
healthy        35
anthracnose    23
Name: count, dtype: int64

In [21]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
33,ID_QEGas6,0,174.999840,316.999680,735.999840,1280.000000,anthracnose
47,ID_TSkjv2,1,554.999580,567.999945,1026.999540,809.999595,cssvd
82,ID_yZDVIT,2,96.000112,1.000064,258.000080,131.000064,healthy
16,ID_ydI3Pa,0,612.998784,515.999232,1562.999040,2442.999552,anthracnose
68,ID_Q9wpbL,2,0.000000,594.000288,2948.998752,3571.999200,healthy


In [22]:
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
3187,ID_RUW7ek.jpg,cssvd,0.000015,288.165131,398.442169,416.000000,415.818970
4231,ID_bJrfR5.jpeg,anthracnose,0.000098,0.442800,0.000000,1809.324341,998.192139
1462,ID_B9K2SI.jpg,anthracnose,0.000017,837.561523,662.879517,960.000000,834.607849
244,ID_Olcex7.jpeg,healthy,0.033546,1851.341797,622.490723,2505.221924,1549.640015
1120,ID_VBvSm2.jpeg,cssvd,0.002249,612.399719,458.952332,956.499512,576.000000


In [23]:
sub.loc[:, "Image_ID"] = sub["Image_ID"].apply(lambda x: str(Path(x).stem))

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
1300,ID_N1BVjv,cssvd,0.000373,0.000000,67.460289,156.064972,940.707092
4221,ID_sFXWTT,healthy,0.001927,218.536728,0.101299,409.847473,56.763058
220,ID_Olcex7,cssvd,0.000488,857.159241,2360.163818,2943.751221,3513.269287


In [24]:
def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels["Image_ID"].unique()}

In [25]:
converted_labels.isna().sum()

Image_ID    0
class_id    0
xmin        0
ymin        0
xmax        0
ymax        0
class       0
dtype: int64

In [26]:
ground_truth = convert_df(converted_labels)
ground_truth = {k: ground_truth[k] for k in converted_labels["Image_ID"].unique()}

len(ground_truth)

56

In [27]:
import torch

def calculate_iou_tensor(box1, box2):
	"""
	box1: [4], box2: [4]
	Format: [xmin, ymin, xmax, ymax]
	"""
	xA = torch.max(box1[0], box2[0])
	yA = torch.max(box1[1], box2[1])
	xB = torch.min(box1[2], box2[2])
	yB = torch.min(box1[3], box2[3])

	inter_area = torch.clamp(xB - xA, min=0) * torch.clamp(yB - yA, min=0)
	box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
	box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
	union_area = box1_area + box2_area - inter_area
	return inter_area / union_area if union_area > 0 else torch.tensor(0.0)

def evaluate_detection(predictions, ground_truths, iou_threshold=0.5, conf_threshold=0.0):
	"""
	predictions: list of dicts (len = batch size), each dict with 'boxes', 'scores', 'labels'
	ground_truths: list of dicts with 'boxes', 'labels'
	"""
	TP = 0
	FP = 0
	FN = 0

	for preds, gts in zip(predictions, ground_truths):
		pred_boxes = preds['boxes']
		pred_labels = preds['labels']
		pred_scores = preds['scores'] if preds['scores'] is not None else torch.ones(len(pred_boxes))

		gt_boxes = gts['boxes']
		gt_labels = gts['labels']
		matched_gt = set()

		for i in range(len(pred_boxes)):
			if pred_scores[i] < conf_threshold:
				continue
			pred_box = pred_boxes[i]
			pred_label = pred_labels[i]
			match_found = False

			for j in range(len(gt_boxes)):
				if j in matched_gt:
					continue
				if pred_label != gt_labels[j]:
					continue
				iou = calculate_iou_tensor(pred_box, gt_boxes[j])
				if iou >= iou_threshold:
					TP += 1
					matched_gt.add(j)
					match_found = True
					break
			if not match_found:
				FP += 1

		FN += len(gt_boxes) - len(matched_gt)

	precision = TP / (TP + FP) if (TP + FP) else 0.0
	recall = TP / (TP + FN) if (TP + FN) else 0.0
	f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
	accuracy = TP / (TP + FP + FN) if (TP + FP + FN) else 0.0

	return {
		'TP': TP,
		'FP': FP,
		'FN': FN,
		'Precision': precision,
		'Recall': recall,
		'F1 Score': f1_score,
		'Accuracy': accuracy
	}

In [28]:
sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
1528,ID_B9K2SI,healthy,0.000033,0.000000,0.000000,66.682495,793.923035
2127,ID_XOgJSK,cssvd,0.000010,1997.923340,398.550049,2048.000000,831.320923
4081,ID_crD9na,cssvd,0.000203,24.094574,2448.000000,3242.602539,2448.000000


In [29]:
sub["class_id"] = sub["class"].map(class_map)

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
3496,ID_T4Tw1d,healthy,0.000026,15.029163,960.000000,390.390930,960.00000,2
3933,ID_E2z7VZ,healthy,0.000083,757.216919,741.302490,1195.034790,960.00000,2
1050,ID_u0KnR0,healthy,0.086196,2269.391602,610.648743,3032.644043,1211.05896,2


In [30]:
predictions = get_preds_data(sub, None)

len(predictions)

56

In [31]:
for i in np.linspace(0.0, 0.95, 15):
	scores = evaluate_detection(
		predictions.values(),
		ground_truth.values(),
		iou_threshold=0.5,
		conf_threshold=i
	)
	print("Evaluation metric at:", i, " score :", scores)

Evaluation metric at: 0.0  score : {'TP': 92, 'FP': 4948, 'FN': 3, 'Precision': 0.018253968253968255, 'Recall': 0.968421052631579, 'F1 Score': 0.03583252190847127, 'Accuracy': 0.018243109260360896}
Evaluation metric at: 0.06785714285714285  score : {'TP': 82, 'FP': 60, 'FN': 13, 'Precision': 0.5774647887323944, 'Recall': 0.8631578947368421, 'F1 Score': 0.6919831223628692, 'Accuracy': 0.5290322580645161}
Evaluation metric at: 0.1357142857142857  score : {'TP': 77, 'FP': 37, 'FN': 18, 'Precision': 0.6754385964912281, 'Recall': 0.8105263157894737, 'F1 Score': 0.736842105263158, 'Accuracy': 0.5833333333333334}
Evaluation metric at: 0.20357142857142857  score : {'TP': 75, 'FP': 23, 'FN': 20, 'Precision': 0.7653061224489796, 'Recall': 0.7894736842105263, 'F1 Score': 0.7772020725388602, 'Accuracy': 0.635593220338983}
Evaluation metric at: 0.2714285714285714  score : {'TP': 68, 'FP': 20, 'FN': 27, 'Precision': 0.7727272727272727, 'Recall': 0.7157894736842105, 'F1 Score': 0.7431693989071039, 'A

In [32]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
73,ID_dOxqJU,1,15.000480,35.000320,804.000480,1280.000000,cssvd
61,ID_crD9na,2,245.999520,812.998752,2218.000608,2774.998944,healthy
3,ID_MYSxE2,2,372.999312,0.000000,1850.000112,3264.000000,healthy
5,ID_OB1kf9,0,115.000320,51.999360,960.000000,1279.999360,anthracnose
15,ID_cmfzof,0,40.000512,214.000640,1280.001024,2046.000128,anthracnose


In [33]:
import torch
from torchmetrics.detection import MeanAveragePrecision


def compute_map(preds, targets):
    # Initialize the metric
    metric = MeanAveragePrecision()

    # Update metric with predictions and targets
    metric.update(preds, targets)

    # Compute the results
    result = metric.compute()

    return result


thrs = np.linspace(0.000001, 0.95, 15)
for i in thrs:
    preds = list(get_preds_data(sub, i).values())

    targets = list(ground_truth.values())

    # Compute mAP
    results = compute_map(preds, targets)

    # Print results
    print("mAP Results:", i, " - ", results)

mAP Results: 1e-06  -  {'map': tensor(0.5764), 'map_50': tensor(0.8186), 'map_75': tensor(0.6430), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.5774), 'mar_1': tensor(0.4530), 'mar_10': tensor(0.6645), 'mar_100': tensor(0.6794), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.6794), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.06785807142857142  -  {'map': tensor(0.5606), 'map_50': tensor(0.7783), 'map_75': tensor(0.6378), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.5610), 'mar_1': tensor(0.4530), 'mar_10': tensor(0.6379), 'mar_100': tensor(0.6379), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.6379), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.13571514285714284  -  {'map': tensor(0.5411), 'map_50': tensor(

In [34]:
sub.to_csv('dataset/evaluations/validation.csv', index=False)

In [35]:
import pandas as pd

sub = pd.read_csv('dataset/evaluations/validation.csv')
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
2658,ID_hRfHto,cssvd,0.000015,6.799992,3024.000000,722.896118,3024.000000,1
1607,ID_yZDVIT,healthy,0.002089,0.000000,378.649231,150.988007,416.000000,2
908,ID_gqbwtB,anthracnose,0.000009,139.071930,353.193146,412.183533,415.790588,0
2581,ID_vdbf6i,healthy,0.003328,3.782997,603.010986,672.968445,956.127625,2
2175,ID_as2hds,anthracnose,0.000125,1072.184937,530.950256,1444.925415,854.114807,0
